In [15]:
import os
import re

In [16]:
pred_dir = "../../datasets/acs-20210530-gold"
target_dir = "../../datasets/acs-20210530-gold-target"

In [17]:
def extract_relations(ann_path):
    res = []
    with open(ann_path, "r") as fin:
        for line in fin:
            items = line.strip().split("\t")
            if items[0].startswith("R"):
                match = re.match("(.*) Arg1:(.*) Arg2:(.*)", items[1])
                res.append({
                    "id": items[0],
                    "type": match[1],
                    "entities": (match[2], match[3])
                })
    return res

In [29]:
len(extract_relations(os.path.join(pred_dir, "sb300091d/sb300091d.ann")))

28

In [34]:
len(extract_relations(os.path.join(target_dir, "sb300091d/sb300091d.ann")))

45

In [52]:
def evaluate(pub_num):
    pred = extract_relations(os.path.join(pred_dir, f"{pub_num}/{pub_num}.ann"))
    target = extract_relations(os.path.join(target_dir, f"{pub_num}/{pub_num}.ann"))
    pred = {x["entities"]: x["type"] for x in pred}
    target = {x["entities"]: x["type"] for x in target}
    not_in_target = 0
    not_found = len(target)
    print(not_found)
    found_correct = 0
    found_incorrect = 0
    for pk, pv in pred.items():
        if pk in target or (pk[1], pk[0]) in target:
            try:
                tv = target[pk]
            except:
                tv = target[(pk[1], pk[0])]
            if pv == tv:
                found_correct += 1
            else:
                found_incorrect += 1
            not_found -= 1
        else:
            not_in_target += 1
    print(f"# of predictions found in target and correct:\t\t{found_correct}")
    print(f"# of predictions found in target but incorrect:\t\t{found_incorrect}")
    print(f"# predictions not found in target (false positive):\t{not_in_target}")
    print(f"# targets not found in prediction (false negative):\t{not_found}")

In [53]:
evaluate("sb300091d")

45
# of predictions found in target and correct:		22
# of predictions found in target but incorrect:		1
# predictions not found in target (false positive):	5
# targets not found in prediction (false negative):	22


In [54]:
evaluate("sb4001382")

151
# of predictions found in target and correct:		78
# of predictions found in target but incorrect:		4
# predictions not found in target (false positive):	34
# targets not found in prediction (false negative):	69
